<a href="https://colab.research.google.com/github/safin777/SmartFarming/blob/main/SmartFarming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading the dataset

In [1]:
# Download a file based on its file ID.
file_id = '18DbC6Xj4NP-hLzI14WuMaAEyq482vNfn'

# Download dataset
!gdown https://drive.google.com/uc?id={file_id}

# Unzip the downloaded file
!unzip -q PlantVillage.zip

Downloading...
From: https://drive.google.com/uc?id=18DbC6Xj4NP-hLzI14WuMaAEyq482vNfn
To: /content/PlantVillage.zip
866MB [00:04, 195MB/s]
replace PlantVillage/train/Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: None


Importing the libraries

In [2]:
import numpy as np
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Load Dataset

Initializing a few parameters required for the image dataset preprocessing

In [3]:
# Dimension of resized image
DEFAULT_IMAGE_SIZE = tuple((256, 256))

# Number of images used to train the model
N_IMAGES = 100

# Path to the dataset folder
root_dir = './PlantVillage'

train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')

We use the function convert_image_to_array to resize an image to the size DEFAULT_IMAGE_SIZE we defined above.

In [4]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, DEFAULT_IMAGE_SIZE)   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None


Here, we load the training data images by traversing through all the folders and converting all the images and labels into separate lists respectively.

NOTE: We use a small portion of the entire dataset due to the computing limitations. Tweak N_IMAGES to include entire dataset.

In [5]:
image_list, label_list =[], []

try: 
  print("[INFO] Loading image......")
  plant_disease_folder_list = listdir(train_dir)

  for plant_disease_folder in plant_disease_folder_list:
    print(f"[INFO] Processing {plant_disease_folder}......")
    plant_disease_image_list = listdir(f"{train_dir}/{plant_disease_folder}/")

    for image in plant_disease_image_list[:N_IMAGES]:
      image_directory = f"{train_dir}/{plant_disease_folder}/{image}"
      if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(plant_disease_folder)

    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

# Transform the loaded training image data into numpy array
np_image_list = np.array(image_list, dtype=np.float16) / 225.0
print()

# Check the number of images loaded for training
image_len = len(image_list)
print(f"Total number of images: {image_len}")


[INFO] Loading image......
[INFO] Processing Tomato___Target_Spot......
[INFO] Image loading completed
[INFO] Processing Blueberry___healthy......
[INFO] Image loading completed
[INFO] Processing Cherry_(including_sour)___Powdery_mildew......
[INFO] Image loading completed
[INFO] Processing Tomato___Late_blight......
[INFO] Image loading completed
[INFO] Processing Pepper,_bell___healthy......
[INFO] Image loading completed
[INFO] Processing Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot......
[INFO] Image loading completed
[INFO] Processing Apple___healthy......
[INFO] Image loading completed
[INFO] Processing Corn_(maize)___healthy......
[INFO] Image loading completed
[INFO] Processing Squash___Powdery_mildew......
[INFO] Image loading completed
[INFO] Processing Corn_(maize)___Northern_Leaf_Blight......
[INFO] Image loading completed
[INFO] Processing Soybean___healthy......
[INFO] Image loading completed
[INFO] Processing Tomato___Early_blight......
[INFO] Image loading complet

Examine the labels/classes in the training dataset.

In [6]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

pickle.dump(label_binarizer,open('plant_disease_label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

print("Total number of classes: ", n_classes)

Total number of classes:  39


#Augment and Split Dataset

Using ImageDataGenerator to augment data by performing various operations on the training images.

In [7]:
augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, 
                             fill_mode="nearest")

Splitting the data into training and test sets for validation purpose.

In [8]:
print("[INFO] Splitting data to train and test...")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Splitting data to train and test...


# Build Model

Defining the hyperparameters of the plant disease classification model.

In [9]:
EPOCHS = 30
STEPS = 100
LR = 1e-3
BATCH_SIZE = 32
WIDTH = 256
HEIGHT = 256
DEPTH = 3

Creating a sequential model and adding Convolutional, Normalization, Pooling, Dropout and Activation layers at the appropriate positions.

In [10]:
model = Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1

if K.image_data_format() == "channels_first":
    inputShape = (DEPTH, HEIGHT, WIDTH)
    chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 85, 85, 64)        0

# Train Model


We initialize Adam optimizer with learning rate and decay parameters.

Also, we choose the type of loss and metrics for the model and compile it for training

In [11]:
# Initialize optimizer
opt = Adam(lr=LR, decay=LR / EPOCHS)

# Compile model
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Train model
print("[INFO] Training network...")
history = model.fit_generator(augment.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              validation_data=(x_test, y_test),
                              steps_per_epoch=len(x_train) // BATCH_SIZE,
                              epochs=EPOCHS, 
                              verbose=1)

[INFO] Training network...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
97/97 [==============================] - 758s 8s/step - loss: 0.1053 - accuracy: 0.2549 - val_loss: 0.3169 - val_accuracy: 0.0474
Epoch 2/30
97/97 [==============================] - 763s 8s/step - loss: 0.0850 - accuracy: 0.3957 - val_loss: 0.3672 - val_accuracy: 0.0372
Epoch 3/30
97/97 [==============================] - 761s 8s/step - loss: 0.0681 - accuracy: 0.5087 - val_loss: 0.3036 - val_accuracy: 0.0526
Epoch 4/30
97/97 [==============================] - 746s 8s/step - loss: 0.0588 - accuracy: 0.5719 - val_loss: 0.1828 - val_accuracy: 0.1949
Epoch 5/30
97/97 [==============================] - 760s 8s/step - loss: 0.0571 - accuracy: 0.5787 - val_loss: 0.2119 - val_accuracy: 0.2244
Epoch 6/30
97/97 [==============================] - 756s 8s/step - loss: 0.0480 - accuracy: 0.6392 - val_loss: 0.0697 - val_accuracy: 0.5603
Epoch 7/30
97/97 [==============================] -

#Evaluate Model
